<a href="https://colab.research.google.com/github/sren97/Proyecto-Modelos/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os

# **Interacción con los Archivos**

## **Descargar archivos CSV**

Con este código se descargan los archivos directamente desde la plataforma Kaggle mediante la API

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


## **Descomprimir los archivos**

In [ ]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


#**Cargar archivos como Dataframes**

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
auxiliar_df = train_df.copy()



# **Limpieza de Datos**


## **Funciones de Limpieza**

### **Rendimiento Global**

In [ ]:
def rendimientoglobal(df):
  map_dict = {
    "alto":3,
    "medio-alto":2,
    "medio-bajo":1,
    "bajo": 0
  }
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(map_dict)

In [ ]:
print(train_df["RENDIMIENTO_GLOBAL"].head(10))




0    medio-alto
1          bajo
2          bajo
3          alto
4    medio-bajo
5    medio-alto
6          alto
7    medio-bajo
8    medio-bajo
9          alto
Name: RENDIMIENTO_GLOBAL, dtype: object


### **Rendimiento Global Reverso**

In [ ]:
def rendimientoglobalreverse(df):
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].round().astype(int)
  map_dict = {
    3:"alto",
    2:"medio-alto",
    1:"medio-bajo",
    0:"bajo"
  }
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(map_dict)

### **Estrato**

In [ ]:
def estrato(df):

  moda = df['FAMI_ESTRATOVIVIENDA'].mode()[0]
  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].apply(generar_valores)


### **Internet**

In [ ]:
#NUEVO - la probabilidad no es una constante, es el promedio de personas que dijieron que si
import random
def internet(df):

  percent = np.round(df['FAMI_TIENEINTERNET'].value_counts(normalize=True).get('Si', 0), 4)

  def assign_value(row):
    est_value=row['FAMI_ESTRATOVIVIENDA']
    if pd.isnull(row['FAMI_TIENEINTERNET']):
        if ((est_value != 'Sin Estrato') or (est_value != 'Estrato 1') or (est_value != 'Estrato 2')) :
            return 'Si'
        elif np.round(random.random(),4) <= percent:
            return 'Si'
        else:
            return 'No'
    else:
        return row['FAMI_TIENEINTERNET']

  df['FAMI_TIENEINTERNET'] = df.apply(assign_value, axis=1)

### **Matricula Propia**

In [ ]:
def matriculapropia(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    moda = df['ESTU_PAGOMATRICULAPROPIO'].mode()[0]

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores_normales(row):
        if pd.isna(row):  # Si el valor es nulo
            return moda
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].apply(generar_valores_normales)


### **Horas que Trabaja**

In [ ]:
def horassemanatrabaja(df):

  moda = df['ESTU_HORASSEMANATRABAJA'].mode()[0]

  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].apply(generar_valores)

### **Valor de la Matricula**

In [ ]:
def valormatriculauniversidad(df):

  moda = df['ESTU_VALORMATRICULAUNIVERSIDAD'].mode()[0]
  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(generar_valores)

### **Educación de Padre**

In [ ]:
def educacionpadre(df):

  moda = df['FAMI_EDUCACIONPADRE'].mode()[0]

  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].apply(generar_valores)


### **Educación de Madre**

In [ ]:
def educacionmadre(df):

  moda = df['FAMI_EDUCACIONMADRE'].mode()[0]
  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].apply(generar_valores)

### **Periodo**

In [ ]:
def periodo(df):

  moda = df['PERIODO'].mode()[0]
  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['PERIODO'] = df['PERIODO'].apply(generar_valores)

### **Departamento**

In [ ]:
def departamento(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    moda = df['ESTU_PRGM_DEPARTAMENTO'].mode()[0]

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores(row):
        if pd.isna(row):  # Si el valor es nulo
            return moda
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].apply(generar_valores)


### **Añadir Nuevas Columnas de Relación**

In [ ]:
def nuevascolumnas(df):
    df['RELACION_PROGRAMA_VALOR_MATRICULA'] = df.apply(lambda x: f"{x['ESTU_PRGM_ACADEMICO']}_{x['ESTU_VALORMATRICULAUNIVERSIDAD']}", axis=1)

    df['RELACION_PROGRAMA_VALOR_MATRICULA_DEPARTAMENTO'] = df.apply(lambda x: f"{x['ESTU_PRGM_ACADEMICO']}_{x['ESTU_VALORMATRICULAUNIVERSIDAD']}_{x['ESTU_PRGM_DEPARTAMENTO']}", axis=1)

    df['RELACION_PROGRAMA_DEPARTAMENTO'] = df.apply(lambda x: f"{x['ESTU_PRGM_ACADEMICO']}_{x['ESTU_PRGM_DEPARTAMENTO']}", axis=1)

    df.drop(columns=['ESTU_PRGM_ACADEMICO'], inplace=True)

### **Programa Acádemico**

In [ ]:
def programaacademico(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    moda = df['ESTU_PRGM_ACADEMICO'].mode()[0]

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores(row):
        if pd.isna(row):  # Si el valor es nulo
            return moda
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(generar_valores)


## **Limpieza Total**

In [ ]:
def limpiar_df(df, predict=False, test=False):
  if not predict:
    if not test:
      rendimientoglobal(df)
    estrato(df)
    internet(df)
    matriculapropia(df)
    valormatriculauniversidad(df)
    educacionpadre(df)
    educacionmadre(df)
    horassemanatrabaja(df)
    departamento(df)
    programaacademico(df)
    periodo(df)
    nuevascolumnas(df)
    return df
  elif predict:
    rendimientoglobal(df)

In [ ]:
limpiar_df(auxiliar_df, predict=True)
train_df2 = limpiar_df(train_df)
test_df2 = limpiar_df(test_df, test=True)

# **Catboost Classifier**

In [ ]:
!pip install catboost


In [ ]:
train_df2.to_csv("train_limpio.csv", index=False)


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar los datos
data = train_df2.copy()  # train_df_clean

# Dividir datos en características y variable objetivo
X = data.drop(columns=["ID", "RENDIMIENTO_GLOBAL"])
y = data["RENDIMIENTO_GLOBAL"]

categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=16, stratify=y)

for col in categorical_features:
    X_train[col] = X_train[col].fillna("Desconocido")
    X_test[col] = X_test[col].fillna("Desconocido")

# Crear el modelo CatBoost con bootstrap_type='Bayesian'
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.07,
    depth=6,
    l2_leaf_reg=3,
    bootstrap_type='Bernoulli',
    subsample=0.75,
    auto_class_weights='Balanced',
    thread_count=4,
    verbose=100,
    task_type='CPU',
    eval_metric='Accuracy',
)

# Entrenar el modelo
model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=categorical_features, use_best_model=True)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy}")

0:	learn: 0.4012192	test: 0.4062662	best: 0.4062662 (0)	total: 9.67s	remaining: 2h 41m
100:	learn: 0.4392322	test: 0.4416953	best: 0.4417687 (97)	total: 15m 10s	remaining: 2h 15m 2s
200:	learn: 0.4436451	test: 0.4451858	best: 0.4453603 (198)	total: 31m 29s	remaining: 2h 5m 10s
300:	learn: 0.4459817	test: 0.4462301	best: 0.4467656 (266)	total: 47m 53s	remaining: 1h 51m 13s
400:	learn: 0.4471478	test: 0.4465645	best: 0.4467656 (266)	total: 1h 3m 43s	remaining: 1h 35m 11s
500:	learn: 0.4483953	test: 0.4466698	best: 0.4467973 (461)	total: 1h 19m 41s	remaining: 1h 19m 22s
600:	learn: 0.4493940	test: 0.4466364	best: 0.4470428 (562)	total: 1h 35m 56s	remaining: 1h 3m 41s
700:	learn: 0.4506922	test: 0.4467923	best: 0.4470428 (562)	total: 1h 51m 42s	remaining: 47m 39s
800:	learn: 0.4519511	test: 0.4464309	best: 0.4470428 (562)	total: 2h 7m 30s	remaining: 31m 40s
900:	learn: 0.4528441	test: 0.4469274	best: 0.4470428 (562)	total: 2h 23m 5s	remaining: 15m 43s
999:	learn: 0.4537468	test: 0.4467094	

# **Predicción Kaggle**

In [ ]:
pred = test_df2.copy()
y = pred["ID"]

# Eliminar columnas innecesarias si existen
columnas_a_eliminar = [col for col in ["ID", "Unnamed: 0"] if col in pred.columns]
pred.drop(columns=columnas_a_eliminar, inplace=True)

# Rellenar NaN en columnas categóricas
for col in pred.select_dtypes(include=['object']).columns:
    pred[col] = pred[col].fillna("Desconocido")

# Predecir
predicciones = model.predict(pred)

# Concatenar resultados
sol = pd.concat([y, pd.DataFrame(predicciones, columns=["RENDIMIENTO_GLOBAL"])], axis=1)

# Convertir de números a etiquetas
rendimientoglobalreverse(sol)

# Guardar a CSV
sol.to_csv("submission.csv", index=False)


In [ ]:
!kaggle competitions submit -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia -f submission.csv -m "Este es el modelo CatBoost"


100% 4.08M/4.08M [00:00<00:00, 5.83MB/s]
Successfully submitted to UDEA/ai4eng 20251 - Pruebas Saber Pro Colombia